In [1]:
import numpy as np
import scipy.io as sci2

import os
#import matplotlib.pyplot as plt
#from matplotlib.font_manager import FontProperties

#%matplotlib inline


from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, save, output_notebook, output_file
from bokeh.layouts import column, gridplot, layout
from bokeh.plotting import figure
from bokeh.models import Div, FixedTicker
from bokeh.palettes import brewer

In [4]:
teste = sci2.loadmat('rat1-verde-WD-20150319-60min-255trials-68PC-SPK_markers_incorrect.mat')
teste

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Sep 22 11:19:57 2016',
 '__version__': '1.0',
 'incorrectTrials': array([[  9.72727750e+01,   0.00000000e+00],
        [  1.25130525e+02,   0.00000000e+00],
        [  1.25130525e+02,   1.00000000e+00],
        [  1.37786375e+02,   0.00000000e+00],
        [  1.37786375e+02,   1.00000000e+00],
        [  1.52878400e+02,   0.00000000e+00],
        [  1.52878400e+02,   1.00000000e+00],
        [  1.97754850e+02,   0.00000000e+00],
        [  2.26930075e+02,   0.00000000e+00],
        [  3.14186225e+02,   0.00000000e+00],
        [  3.85580975e+02,   0.00000000e+00],
        [  3.99643825e+02,   0.00000000e+00],
        [  4.58000200e+02,   0.00000000e+00],
        [  4.85772825e+02,   0.00000000e+00],
        [  4.85772825e+02,   1.00000000e+00],
        [  5.50914750e+02,   0.00000000e+00],
        [  5.81056275e+02,   0.00000000e+00],
        [  5.81056275e+02,   1.00000000e+00],
        [  5.9

In [5]:

output_notebook()

x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)

p = figure(title="simple line example", plot_height=300, plot_width=600, y_range=(-5,5))
r = p.line(x, y, color="#2222aa", line_width=3)

def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    elif f == "tan": func = np.tan
    r.data_source.data['y'] = A * func(w * x + phi)
    push_notebook()

show(p, notebook_handle=True)
interact(update, f=["sin", "cos", "tan"], w=(0,100), A=(1,5), phi=(0, 20, 0.1))

<function __main__.update>

In [5]:
all_rats = sorted(os.listdir('learning-curves'))
x = list(np.arange(-4,3.525,0.025))
output_file('F1-score_all_rats_all_and_top30_attributes.html')
#output_notebook()
plots = []
subplot_width = 800 
for rat in all_rats:
    rat_performance_files = sorted(os.listdir('learning-curves/' + rat), reverse=True)
    line_plots = []
    for file in rat_performance_files:
        rat_performance = np.loadtxt('learning-curves/' + rat + '/' + file, delimiter='\n', dtype=bytes).astype(float)
        titleFig = rat
        if(file.startswith('better')):
            titleFig = titleFig + '_top30'
        p = figure(title=titleFig, plot_height=300, plot_width=subplot_width, y_range=(0.35,1), x_range=(-4.5,4.5))
        p.xaxis[0].axis_label = 'Time(s)'
        p.yaxis[0].axis_label = 'F1-score'
        p.xaxis[0].ticker=FixedTicker(ticks=np.arange(-4,4.5,0.5))
        r = p.line(x, rat_performance, color="#2222aa", line_width=1)
        line_plots.append(p)
    plots.append(line_plots)
#show(p)
#não dá para colocar titulo em um gridplot, mas abaixo eis a gambi. É possível compor um gridplot com uma Div html
save(column(Div(width=2*subplot_width, text = '<h2 style="text-align:center;">Curvas (F1-score) utilizando apenas spikes</h2>'),gridplot(plots)))

'C:\\TCC\\F1-score_all_rats_all_and_top30_attributes.html'

In [2]:
all_rats = sorted(os.listdir('spikes_right'))
top = 30
#guardará para cada animal, a evolução do histograma de tipos de neurônios no top 'top' 
all_neurons_type_hists_by_animal = {}
all_neurons_attributeType_hists_by_animal = {}
all_neurons_regions_hists_by_animal = {}

for rat in all_rats:
    rat_spikes = sci2.loadmat('spikes_right/' + rat)
    
    rat_neurons = list(key for key in rat_spikes.keys() 
                           if key.startswith('SI')  or 
                              key.startswith('PFC') or 
                              key.startswith('VI')  or 
                              key.startswith('PPC'))
    
    if(rat.startswith('rat3-preto')):
        continue    #rato 3 está faltando um neuronio no VI ou renan mandou um neuronio a mais
        print(list(key for key in rat_spikes.keys() 
                               if not key.startswith('SI')  and not 
                                  key.startswith('PFC') and not
                                  key.startswith('VI')  and not 
                                  key.startswith('PPC')))
        
    rat_neurons_PFC = sorted(list(key for key in rat_spikes.keys() if key.startswith('PFC')))
    rat_neurons_PPC = sorted(list(key for key in rat_spikes.keys() if key.startswith('PPC')))
    rat_neurons_SI = sorted(list(key for key in rat_spikes.keys() if key.startswith('SI')))
    rat_neurons_VI = sorted(list(key for key in rat_spikes.keys() if key.startswith('VI')))
                              
    
    rat_neurons_labels = sci2.loadmat('tipo_de_resposta_neuronios/' + rat.split('_')[0] + '_responseType.mat')['responseType']
    rat_neurons_labels_PFC = rat_neurons_labels[0, 0][0]
    rat_neurons_labels_PPC = rat_neurons_labels[0, 1][0]
    rat_neurons_labels_SI = rat_neurons_labels[0, 2][0]
    rat_neurons_labels_VI = rat_neurons_labels[0, 3][0]
    
    #construindo dicionário neuronio -> tipo
    neuron_name_label_mapping = {}
    
    for i in range(0, len(rat_neurons_PFC)):
        neuron_name_label_mapping[rat_neurons_PFC[i]] = rat_neurons_labels_PFC[i]
    for i in range(0, len(rat_neurons_PPC)):
        neuron_name_label_mapping[rat_neurons_PPC[i]] = rat_neurons_labels_PPC[i]
    for i in range(0, len(rat_neurons_SI)):
        neuron_name_label_mapping[rat_neurons_SI[i]] = rat_neurons_labels_SI[i]
    for i in range(0, len(rat_neurons_VI)):
        neuron_name_label_mapping[rat_neurons_VI[i]] = rat_neurons_labels_VI[i]
    
    #lendo arquivo de rankings de neurônios para esse animal na discriminação tátil
    all_ranking_files = os.listdir('ranking/' + rat)
    temp_to_order_mapping = {}
    for r_file in all_ranking_files:
        temp_to_order_mapping[float(r_file.split('_')[3].split('@')[0])] = r_file
    
    files_keys_ordered = sorted(temp_to_order_mapping.keys())
    
    #guardará para esse animal, o numero de ocorrencias de neuronios de cada tipo no top 'top' no decorrer do experimento
    all_neurons_type_hists = {} 
    all_neurons_attributeType_hists = {}
    all_neurons_regions_hists = {}
    
    for f in files_keys_ordered:
        neuron_rankings = np.loadtxt('ranking/' + rat + '/' + temp_to_order_mapping[f], delimiter='\n', dtype=bytes).astype(str)
        picked = 0
        index = 0
        top_neurons_type_hist = {}
        top_neurons_type_hist[0] = 0
        top_neurons_type_hist[1] = 0
        top_neurons_type_hist[2] = 0
        top_neurons_type_hist[3] = 0
        
        top_neurons_attributeType_hist = {}
        top_neurons_attributeType_hist["lat"] = 0
        top_neurons_attributeType_hist["num"] = 0
        
        top_neurons_regions_hist = {}
        top_neurons_regions_hist["SI"] = 0
        top_neurons_regions_hist["VI"] = 0
        top_neurons_regions_hist["PPC"] = 0
        top_neurons_regions_hist["PFC"] = 0
        
        #pick 'top' best attributes to check statistics
        while(picked < top):
            neuron = neuron_rankings[index].split('_')[0]
            neuron_attribute_type = neuron_rankings[index].split('_')[1]    #identifica se esse atributo é de número ou é de latência de spikes
            if(neuron in neuron_name_label_mapping):   #esse if era útil no caso dos atributos de lfp estarem o top 30 (eles eram pulados, já que os atributos de lfps não estão relacionado a um neurônio específico)
                top_neurons_type_hist[neuron_name_label_mapping[neuron]] += 1
                top_neurons_attributeType_hist[neuron_attribute_type] += 1
                if(neuron.startswith("SI")):
                    top_neurons_regions_hist["SI"] += 1
                elif(neuron.startswith("VI")):
                    top_neurons_regions_hist["VI"] += 1
                elif(neuron.startswith("PPC")):
                    top_neurons_regions_hist["PPC"] += 1
                elif(neuron.startswith("PFC")):
                    top_neurons_regions_hist["PFC"] += 1
                picked += 1
            index += 1
        
        all_neurons_type_hists[f] = top_neurons_type_hist
        all_neurons_attributeType_hists[f] = top_neurons_attributeType_hist
        all_neurons_regions_hists[f] = top_neurons_regions_hist
        
    all_neurons_type_hists_by_animal[rat] = all_neurons_type_hists
    all_neurons_attributeType_hists_by_animal[rat] = all_neurons_attributeType_hists
    all_neurons_regions_hists_by_animal[rat] = all_neurons_regions_hists
    
    #np.loadtxt('ranking/rat1-verde-WD-20150319-60min-255trials-68PC-SPK_data_right.mat/ranking_features_0.0@0.5.csv', delimiter='\n', dtype=bytes).astype(str)
    
    print("Rato:", rat)
    print("Numero de neuronios total:", len(rat_neurons))
    print("Numero de neuorions SI", len(rat_neurons_SI), "Numero de labels SI", len(rat_neurons_labels_SI))
    print("Numero de neuorions VI", len(rat_neurons_VI), "Numero de labels VI", len(rat_neurons_labels_VI))
    print("Numero de neuorions PPC", len(rat_neurons_PPC), "Numero de labels PPC", len(rat_neurons_labels_PPC))
    print("Numero de neuorions PFC", len(rat_neurons_PFC), "Numero de labels PFC", len(rat_neurons_labels_PFC))

Rato: rat1-verde-WD-20150319-60min-255trials-68PC-SPK_data_right.mat
Numero de neuronios total: 63
Numero de neuorions SI 31 Numero de labels SI 31
Numero de neuorions VI 4 Numero de labels VI 4
Numero de neuorions PPC 22 Numero de labels PPC 22
Numero de neuorions PFC 6 Numero de labels PFC 6
Rato: rat1-vermelho-WD-20150324-61min-278trials-69PC-SPK_data_right.mat
Numero de neuronios total: 55
Numero de neuorions SI 22 Numero de labels SI 22
Numero de neuorions VI 4 Numero de labels VI 4
Numero de neuorions PPC 25 Numero de labels PPC 25
Numero de neuorions PFC 4 Numero de labels PFC 4
Rato: rat2-preto-WD-20150326-60min-263trials-79PC-SPK_data_right.mat
Numero de neuronios total: 86
Numero de neuorions SI 12 Numero de labels SI 12
Numero de neuorions VI 23 Numero de labels VI 23
Numero de neuorions PPC 23 Numero de labels PPC 23
Numero de neuorions PFC 28 Numero de labels PFC 28
Rato: rat3-azul-WD-20150319-61min-131trials-82PC-SPK_data_right.mat
Numero de neuronios total: 79
Numero de 

In [8]:
all_neurons_attributeType_hists_by_animal['rat1-verde-WD-20150319-60min-255trials-68PC-SPK_data_right.mat'][-4]['lat']

13

In [9]:
type_name = {}
type_name[0] = 'Invariante'
type_name[1] = 'Excitatório'
type_name[2] = 'Inibitório'
type_name[3] = 'Excitatório<->Inibitório'

color = {}
color[0] = '#995005'
color[1] = '#04780C'
color[2] = '#990905'
color[3] = '#064B5F'

#fontP = FontProperties()
#fontP.set_size('small')   
subplot_width = 800
output_file('pies.html')
all_plots = []
line_plot_colors = brewer['Set1'][4]
for myk in all_neurons_type_hists_by_animal:
    neurons_type_hist = all_neurons_type_hists_by_animal[myk]
    times = sorted(neurons_type_hist.keys())
    
    p = figure(title=myk.split('_')[0], plot_height=250, plot_width=subplot_width, y_range=(0,30), x_range=(-4.5,4.5))
    p.xaxis[0].axis_label = 'Time(s)'
    p.yaxis[0].axis_label = 'Occurrence on top 30'
    p.xaxis[0].ticker=FixedTicker(ticks=np.arange(-4,4.5,0.5))
    p.title.text_font_size = '14pt'
    p.title.text_color = 'blue'
    
    c = 0
    for k in type_name.keys():
        curve = []
        for t in times:
            curve.append(neurons_type_hist[t][k])
        
        r = p.line(times, curve, legend=type_name[k], line_width=2, color=line_plot_colors[c])
        c += 1
    all_plots.append([p])
    
    #pie charts latencia x spikes no top30 nos tempos -2s, -1s, -0.5s, 0s, 0.5s, 1s, 2s
    neurons_attributeType_hist = all_neurons_attributeType_hists_by_animal[myk]
    neurons_regions_hist = all_neurons_regions_hists_by_animal[myk]
    
    pies = []
    for t in [-1, -0.5, 0.5, 1]:
        p = figure(title=myk.split('_')[0], plot_height=200, plot_width=200)
        attrs = sorted(neurons_attributeType_hist[t].keys())
        shares = []
        starts = []
        ends = []
        for attr in attrs:
            shares.append(neurons_attributeType_hist[t][attr])
        
        shares = np.array(shares)
        shares = shares/np.sum(shares)
        shares = list(shares)
        
        cumulative = 0
        percents = []
        s_index = 0
        while(cumulative <= 1):
            percents.append(cumulative)
            if(s_index > len(shares) - 1):
                break
            cumulative += shares[s_index]
            s_index += 1
        starts = [p*2*np.pi for p in percents[:-1]]
        ends = [p*2*np.pi for p in percents[1:]]
        # a color for each pie piece
        colors = ["blue", "orange"]
        
        c = 0
        p = figure(title = "latencia(lat) x spikes(num) at " + str(t) + "s", x_range=(-1,1), y_range=(-1,1))
        #for attr in attrs:
        for aux in range(0, len(starts)):
            p.wedge(x=0, y=0, radius=1, start_angle=starts[aux], end_angle=ends[aux], color=line_plot_colors[aux], legend=attrs[aux] + " (" + str(neurons_attributeType_hist[t][attrs[aux]]) + ")")
        pies.append(p)
    all_plots.append(pies)    
    
    pies = []
    for t in [-1, -0.5, 0.5, 1]:
        p = figure(title=myk.split('_')[0], plot_height=200, plot_width=200)
        attrs = sorted(neurons_regions_hist[t].keys())
        shares = []
        starts = []
        ends = []
        for attr in attrs:
            shares.append(neurons_regions_hist[t][attr])
        
        shares = np.array(shares)
        shares = shares/np.sum(shares)
        shares = list(shares)
        
        cumulative = 0
        percents = []
        s_index = 0
        while(cumulative <= 1):
            percents.append(cumulative)
            if(s_index > len(shares) - 1):
                break
            cumulative += shares[s_index]
            s_index += 1
        starts = [p*2*np.pi for p in percents[:-1]]
        ends = [p*2*np.pi for p in percents[1:]]
        # a color for each pie piece
        colors = ["blue", "orange", "red", "yellow"]
        
        c = 0
        p = figure(title = "laatencia(lat) x spikes(num) at " + str(t) + "s", x_range=(-1,1), y_range=(-1,1))
        #for attr in attrs:
        for aux in range(0, len(starts)):
            p.wedge(x=0, y=0, radius=1, start_angle=starts[aux], end_angle=ends[aux], color=line_plot_colors[aux], legend=attrs[aux] + " (" + str(neurons_regions_hist[t][attrs[aux]]) + ")")
        pies.append(p)
    all_plots.append(pies)
    
    l = layout(all_plots, responsive = True)
save(l)

'C:\\TCC\\pies.html'

In [14]:
all_rats = sorted(os.listdir('correct-incorrect-curves'))
x = list(np.arange(-4,3.525,0.025))
output_file('F1-score_correct_incorrect_all_rats.html')
#output_notebook()
plots = []
subplot_width = 800 
for rat in all_rats:
    rat_performance_file = os.listdir('correct-incorrect-curves/' + rat)
    line_plots = []
    for file in rat_performance_file:
        rat_performance = np.loadtxt('correct-incorrect-curves/' + rat + '/' + file, delimiter='\n', dtype=bytes).astype(float)
        titleFig = rat        
        p = figure(title=titleFig, plot_height=300, plot_width=subplot_width, y_range=(0,1), x_range=(-4.5,4.5))
        p.xaxis[0].axis_label = 'Time(s)'
        p.yaxis[0].axis_label = 'F1-score'
        p.xaxis[0].ticker=FixedTicker(ticks=np.arange(-4,4.5,0.5))
        r = p.line(x, rat_performance, color="#009874", line_width=1)
        line_plots.append(p)
    plots.append(line_plots)
#show(p)
#não dá para colocar titulo em um gridplot, mas abaixo eis a gambi. É possível compor um gridplot com uma Div html
save(column(Div(width=2*subplot_width, text = '<h2 style="text-align:center;">Curvas (F1-score) do modelo treinado com acertos tentando prever os trials errados utilizando apenas spikes</h2>'),gridplot(plots)))

'C:\\TCC\\F1-score_correct_incorrect_all_rats.html'